In [85]:
import intake,yaml
import intake_esm
#anonymous True access to test data s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1/v20190726

In [76]:
storage_options = {'anon': True}

In [77]:
col_url = "/home/jovyan/gfdl-aws-analysis/CatalogBuilder/gfdltest.json" 

In [78]:
col = intake.open_esm_datastore(col_url)

In [79]:
col.df


,product_id,institute,model,experiment,frequency,modeling_realm,mip_table,ensemble_member,variable,temporal_subset,version,path
0,output,NOAA-GFDL,GFDL-ESM4,historical,mon,atmos,Amon,r1i1p1,tas,195001-201412,v20190726,s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...


In [80]:
expname_filter = ['historical']
table_id_filter = 'Amon'
model_filter = 'GFDL-ESM4'
variable_id_filter = "tas"

In [81]:
cat = col.search(experiment=expname_filter, mip_table=table_id_filter,model=model_filter)


In [72]:
type(cat.df.head(1))
type(cat)
cat.df.head(1)


,product_id,institute,model,experiment,frequency,modeling_realm,mip_table,ensemble_member,variable,temporal_subset,version,path
0,output,NOAA-GFDL,GFDL-ESM4,historical,mon,atmos,Amon,r1i1p1,tas,195001-201412,v20190726,s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...


In [82]:
data_source = col["output.NOAA-GFDL.GFDL-ESM4.atmos.historical.mon.Amon"]

In [83]:
print(type(data_source))
print(data_source)

<class 'intake_esm.source.ESMGroupDataSource'>
<name: output.NOAA-GFDL.GFDL-ESM4.atmos.historical.mon.Amon, assets: 1


In [84]:
ds = data_source(zarr_kwargs={'consolidated': True, 'decode_times': True}).to_dask()
ds

OSError: 
            Failed to open netCDF/HDF dataset.

            *** Arguments passed to xarray.open_dataset() ***:

            - filename_or_obj: <fsspec.mapping.FSMap object at 0x7fc9727ad2b0>
            - kwargs: {'chunks': {}}

            *** fsspec options used ***:

            - root: gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1/v20190726
            - protocol: ('s3', 's3a')

            ********************************************
            